참조 : https://github.com/hunkim/DeepLearningZeroToAll
## 연속되는 문장에 대해 각 문자뒤에 나올 문자 예측 (RNN)
- h i h e l l  => i h e l l o 예측

In [38]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(777)

idx2char = ['h','i','e','l','o']
# 다음에 나오는 단어를 학습 hihell => 각 char은 다음 단어들로 ihello가 나오게
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3
y_data = [[1,0,2,3,3,4]] # ihello

num_classes = 5 # 단어 사전 수
input_dim = 5 # one-hot size
hidden_size = 5 # output 사이즈 (one-hot)
batch_size = 1 # 들어가는 문장 (여기선 hielo 하나 뿐)
sequence_length = 6 # |hihell| == 6
learning_rate = 0.01
#==============================
# RNN 모델
#==============================
# X one-hot [batch_size, 들어가게되는 input length(sequence data), 각 input에 대한 차원(one-hot)]
X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])
# Y Label
Y = tf.placeholder(tf.int32, [None, sequence_length]) 
# cell을 생성, num_units는 출력 수
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
# lstn으로하려면 위의 BasicRNNCell을
# cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_id_tuple = True)로 바꿔주면 됨
# 여기서 num_units은 LSTM셀의 output size
# state_is_tuple은 True일 경우 c_state, m_state을 tuple 형식으로, false면 합쳐서 리턴
# cell state를 의미 ,,c_state = (sigmoid(f + self._forget_bias) * c_prev + sigmoid(i) * self._activation(j))
# hidden state를 의미,, m_state = sigmoid(o) * self._activation(c)
# 또는 rnn_cell.GRUCell(rnn_size)를 써서 GRU로도 사용 가능

# 모든 rnn cell들의 값들 0으로 set
initial_state = cell.zero_state(batch_size, tf.float32)
# output tensor와 최종상태인 state 리턴
# outpts = [1,seq_size,output_size]
# _staets = [1,output_size]
# cell 실행
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state = initial_state, dtype = tf.float32)
print("X.shape : {}".format(X.shape))
print("Y.shape : {}".format(Y.shape))
print("outputs.shape : {}".format(outputs.shape))
print("_states.shape : {}".format(_states.shape))
print("====FC layer=====")

#==============================
# FC layer
#==============================

x_for_fc = tf.reshape(outputs, [-1, hidden_size])
print("x_for_fc.shape : {}".format(x_for_fc.shape))
# fc_w = tf.get_variable("fc_w", [hidden_size, num_classes])
# fc_b = tf.get_variable("fc_b", [num_classes])
# outputs = tf.matmul(X_for_fc, fc_w) + fc_b
outputs = tf.contrib.layers.fully_connected(inputs=x_for_fc, num_outputs = num_classes, activation_fn =None)
print(outputs.shape)
#==============================
# loss function
#==============================
# 전체 sequence에 대한 lossfunction 함수가 있음,  
# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
print(outputs.shape)
# 예측된 전체 sequence_length(6)과 label값을 비교해 오차율로 학습
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = Y, weights = weights)

loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

#==============================
# learning 
#==============================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(200):
        l, _ = sess.run([loss, train], feed_dict = {X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict = {X: x_one_hot})
        
        if i % 40 == 0:
            print(i, "loss : {} prediction : {} true Y : {}".format(l, result, y_data) )
            result_str = [idx2char[c] for c in np.squeeze(result)]
            print("\tPredction str : {}".format(''.join(result_str)))

X.shape : (?, 6, 5)
Y.shape : (?, 6)
outputs.shape : (1, 6, 5)
_states.shape : (1, 5)
====FC layer=====
x_for_fc.shape : (6, 5)
(6, 5)
(1, 6, 5)
0 loss : 1.7141227722167969 prediction : [[2 4 1 3 3 3]] true Y : [[1, 0, 2, 3, 3, 4]]
	Predction str : eoilll
40 loss : 0.6970529556274414 prediction : [[2 0 2 3 3 4]] true Y : [[1, 0, 2, 3, 3, 4]]
	Predction str : ehello
80 loss : 0.17991243302822113 prediction : [[1 0 2 3 3 4]] true Y : [[1, 0, 2, 3, 3, 4]]
	Predction str : ihello
120 loss : 0.06610546261072159 prediction : [[1 0 2 3 3 4]] true Y : [[1, 0, 2, 3, 3, 4]]
	Predction str : ihello
160 loss : 0.03839494287967682 prediction : [[1 0 2 3 3 4]] true Y : [[1, 0, 2, 3, 3, 4]]
	Predction str : ihello


##### 3차원 argmax
- argmax 하는 데이터의 shape이 (3, 4, 5)라 한다면
    - axis 0은 0번째를 축으로 하니 결과 값의 형태는 (4, 5)
    - axis 1은 1번째를 축으로 하니 결과 값의 형태는 (3, 5)
    - axis 2은 2번째를 축으로 하니 결과 값의 형태는 (3, 4)


In [70]:
# 3차원 argmax
# axis 0 
# axis 1
# axis 2
test1 = [
         [
             [7,2,3],[8,5,6]
         ],
         [
             [4,5,6],[0,3,9]
         ],
         [
             [4,0,6],[0,7,9]
         ],
         [
             [0,0,6],[0,-7,9]
         ]
       ]

sess = tf.Session()
print("===========")
print(sess.run(tf.argmax(test1, axis =0)))
print("===========")
print(sess.run(tf.argmax(test1, axis =1)))
print("===========")
print(sess.run(tf.argmax(test1, axis =2)))
print("===========")

[[0 1 1]
 [0 2 1]]
[[1 1 1]
 [0 0 1]
 [0 1 1]
 [0 0 1]]
[[0 0]
 [2 2]
 [2 2]
 [2 2]]


In [74]:
# 3차원 argmax
# axis 0 
# axis 1
# axis 2
test1 = [
         [
             [7,2,3],[8,5,6]
         ],
         [
             [4,5,6],[0,3,9]
         ],
         [
             [4,0,6],[0,7,9]
         ],
         [
             [0,0,6],[0,-7,9]
         ]
       ]

sess = tf.Session()
print("===========")
print(sess.run(tf.argmax(test1, axis =0)))
print("===========")
print(sess.run(tf.argmax(test1, axis =1)))
print("===========")
print(sess.run(tf.argmax(test1, axis =2)))
print("===========")

[[0 1 1]
 [0 0 1]]
[[1 1 1]
 [0 0 1]]
[[0 0]
 [2 2]]


## 연속되는 문장에 대해 문자 뒤 나오는 문자 예측 (LSTM)
- one-hot 인코딩을 tensorflow에서 제공하는 함수로 구현
- tf.contrib.rnn.BasicLSTMCell 함수 통해 LSTM모델 사용

In [14]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(777)

sample = " if you want you"
idx2char = list(set(sample))
char2idx = {c: i for i,c in enumerate(idx2char)}

dic_size = len(char2idx) # one-hot size
hidden_size = len(char2idx) # output size
num_classes = len(char2idx) # final output size
batch_size = 1
sequence_length = len(sample) - 1 
learning_rate = 0.1

# 각 문자들을 숫자로
sample_idx = [char2idx[c] for c in sample]
# x data는 0 ~ n-1
x_data = [sample_idx[:-1]] 
# y data는 1 ~ n
y_data = [sample_idx[1:]]

# ================
# LSTM 모델 설계
# ================
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

# one hot으로 바꿔줌
x_one_hot = tf.one_hot(X, num_classes)
print("X.shape : {}".format(X.shape))
print("x_one_hot.shape : {}".format(x_one_hot.shape))
# LSTM cell 생성
# output_size가 hidden_size (단어 one-hot 차원 ), cel state와 hidden state를 tuple 형식으로 반환하는 cell
cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple=True)
# 모든 cell state는 0으로 초기화 하고
initial_state = cell.zero_state(batch_size, tf.float32)
# cell 수행
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)
print("outputs.shape : {}".format(outputs.shape))
# ================
# FC layer
# ================
print("=============== FC layer ===========")
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
print("outputs.shape : {}".format(outputs.shape))
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)
print("outputs.shape : {}".format(outputs.shape))

# ================
# sequence loss 
# ================
outputs = tf.reshape



X.shape : (?, 15)
x_one_hot.shape : (?, 15, 10)
outputs.shape : (1, 15, 10)
=============== FC layer ===========
outputs.shape : (1, 15, 10)
outputs.shape : (15, 10)
